In [1]:
print('start l2 activity')
import sys
sys.path.append("../helper_functions")
import duneapi_utils as d
import growthepieapi_utils as gtp
import l2beat_utils as ltwo
sys.path.pop()

import numpy as np
import pandas as pd

start l2 activity


In [2]:
# # # Usage
gtp_api = gtp.get_growthepie_api_data()
# # print(merged_df.sample(5))
gtp_meta_api = gtp.get_growthepie_api_meta()

In [3]:
l2beat_df = ltwo.get_all_l2beat_data()
l2beat_meta = ltwo.get_l2beat_metadata()
l2beat_meta['chain'] = l2beat_meta['slug']

/Users/michaelsilberling/Documents/GitHub/op-analytics/other_chains_tracking/../helper_functions/l2beat_utils.py:181: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df, pd.DataFrame(data_list)], ignore_index=True)
/Users/michaelsilberling/Documents/GitHub/op-analytics/other_chains_tracking/../helper_functions/l2beat_utils.py:181: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df, pd.DataFrame(data_list)], ignore_index=True)


In [4]:
# print(l2beat_df[l2beat_df['chain']=='degen'].sample(5))
# print(l2beat_meta[l2beat_meta['layer']=='layer2'].sample(5))
# print(l2beat_meta[l2beat_meta['layer']=='layer3'].sample(5))

In [5]:
# display(l2beat_meta.sample(5))

In [6]:
combined_l2b_df = l2beat_df.merge(l2beat_meta[['chain','name','layer','chainId','provider','category','is_upcoming']], on='chain',how='left')
# combined_l2b_df.tail(5)

In [7]:
combined_gtp_df = gtp_api.merge(gtp_meta_api[['origin_key','chain_name']], on='origin_key',how='left')
# combined_gtp_df.sample(5)

In [8]:
# Check Columns
# Assuming combined_gtp_df is your DataFrame
column_names = combined_gtp_df.columns

for col in column_names:
    if col.endswith('_usd'):
        # Construct the new column name by replacing '_usd' with '_eth'
        new_col_name = col.replace('_usd', '_eth')
        
        # Check if the new column name exists in the DataFrame
        if new_col_name not in combined_gtp_df.columns:
            # If it doesn't exist, create the column and fill it with nan values
            combined_gtp_df[new_col_name] = np.nan

In [9]:
# print(combined_gtp_df.dtypes)
# print(l2beat_df.dtypes)
# combined_gtp_df.sample(5)
combined_l2b_df.sample(5)

,timestamp,valueUsd,cbvUsd,ebvUsd,nmvUsd,valueEth,cbvEth,ebvEth,nmvEth,chain,transactions,ethereumTransactions,name,layer,chainId,provider,category,is_upcoming
66033,2022-07-10,9.140971e+08,9.140971e+08,0.0,0.0,7.506215e+05,7.506215e+05,0.0,0.0,ronin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50481,2024-05-15,4.596324e+08,4.596324e+08,0.0,0.0,1.595513e+05,1.595513e+05,0.0,0.0,lzomnichain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
970,2023-10-13,3.764761e+07,3.764761e+07,0.0,0.0,2.449720e+04,2.449720e+04,0.0,0.0,apex,89461.0,1007291.0,ApeX,L2,None,StarkEx,Validium,False
27775,2023-06-05,5.609958e+07,5.609958e+07,0.0,0.0,2.966304e+04,2.966304e+04,0.0,0.0,deversifi,1691.0,1151611.0,NaN,NaN,NaN,NaN,NaN,NaN
52298,2022-08-13,2.418696e+09,2.418696e+09,0.0,0.0,1.234450e+06,1.234450e+06,0.0,0.0,near,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Add Metadata
opstack_metadata = opstack_metadata = pd.read_csv('../op_chains_tracking/outputs/chain_metadata.csv')
combined_l2b_df['l2beat_slug'] = combined_l2b_df['chain']
meta_cols = ['l2beat_slug', 'is_op_chain','mainnet_chain_id','op_based_version', 'alignment','chain_name', 'display_name']

l2b_enriched_df = combined_l2b_df.merge(opstack_metadata[meta_cols], on='l2beat_slug', how = 'left')

l2b_enriched_df['alignment'] = l2b_enriched_df['alignment'].fillna('Other EVMs')
l2b_enriched_df['is_op_chain'] = l2b_enriched_df['is_op_chain'].fillna(False)

In [11]:
#  Define aggregation functions for each column
aggregations = {
    'valueUsd': ['min', 'last', 'mean'],
    'transactions': ['sum', 'mean'],
    'cbvUsd': ['min', 'last', 'mean'],
    'ebvUsd': ['min', 'last', 'mean'],
    'nmvUsd': ['min', 'last', 'mean'],
}

# Group by month, chain, layer, and other specified columns and apply aggregations
l2b_monthly_df = l2b_enriched_df.groupby([pd.Grouper(key='timestamp', freq='MS'), 'chain', 'layer', 'is_op_chain', 'mainnet_chain_id', 'op_based_version', 'alignment', 'chain_name','display_name','provider','is_upcoming'], dropna=False).agg(aggregations).reset_index()

# Flatten the hierarchical column index and concatenate aggregation function names with column names
l2b_monthly_df.columns = [f'{col}_{func}' if func != '' else col for col, func in l2b_monthly_df.columns]
# Rename the 'date' column
l2b_monthly_df.rename(columns={'timestamp': 'month'}, inplace=True)
# Group by 'chain' and rank the rows within each group based on the 'date' column
l2b_monthly_df['months_live'] = l2b_monthly_df.groupby('chain')['month'].rank(method='min')
l2b_monthly_df.sample(5)

,month,chain,layer,is_op_chain,mainnet_chain_id,op_based_version,alignment,chain_name,display_name,provider,...,cbvUsd_min,cbvUsd_last,cbvUsd_mean,ebvUsd_min,ebvUsd_last,ebvUsd_mean,nmvUsd_min,nmvUsd_last,nmvUsd_mean,months_live
2207,2023-12-01,portal,NaN,False,NaN,NaN,Other EVMs,NaN,NaN,NaN,...,2.209454e+08,2.443434e+08,2.417042e+08,0.0,0.0,0.0,0.0,0.0,0.0,28.0
180,2021-02-01,harmony,NaN,False,NaN,NaN,Other EVMs,NaN,NaN,NaN,...,2.649382e+04,7.545312e+04,5.800444e+04,0.0,0.0,0.0,0.0,0.0,0.0,5.0
1062,2022-09-01,polygon-pos,NaN,False,NaN,NaN,Other EVMs,NaN,NaN,NaN,...,2.903338e+09,2.916897e+09,3.317729e+09,0.0,0.0,0.0,0.0,0.0,0.0,26.0
2561,2024-04-01,aztec,L2,False,NaN,NaN,Other EVMs,NaN,NaN,NaN,...,3.707187e+06,3.983403e+06,4.045527e+06,0.0,0.0,0.0,0.0,0.0,0.0,38.0
2656,2024-04-01,xdai,NaN,False,NaN,NaN,Other EVMs,NaN,NaN,NaN,...,3.180855e+05,1.000406e+06,8.858776e+05,0.0,0.0,0.0,0.0,0.0,0.0,67.0


In [12]:
# export
folder = 'outputs/'
combined_gtp_df.to_csv(folder + 'growthepie_l2_activity.csv', index = False)
gtp_meta_api.to_csv(folder + 'growthepie_l2_metadata.csv', index = False)
l2b_enriched_df.to_csv(folder + 'l2beat_l2_activity.csv', index = False)
l2beat_meta.to_csv(folder + 'l2beat_l2_metadata.csv', index = False)
l2b_monthly_df.to_csv(folder + 'l2beat_l2_activity_monthly.csv', index = False)
# Post to Dune API
d.write_dune_api_from_pandas(combined_gtp_df, 'growthepie_l2_activity',\
                             'L2 Usage Activity from GrowThePie')
d.write_dune_api_from_pandas(gtp_meta_api, 'growthepie_l2_metadata',\
                             'L2 Metadata from GrowThePie')
d.write_dune_api_from_pandas(l2b_enriched_df, 'l2beat_l2_activity',\
                             'L2 Usage Activity from L2Beat')
d.write_dune_api_from_pandas(l2b_monthly_df, 'l2beat_l2_activity_monthly',\
                             'Monthly L2 Usage Activity from L2Beat')
d.write_dune_api_from_pandas(l2beat_meta, 'l2beat_l2_metadata',\
                             'L2 Metadata from L2Beat')


table at: dune.oplabspbc.dataset_growthepie_l2_activity
Response status code: 200
Response content: b'{"success":true,"table_name":"growthepie_l2_activity"}'
table at: dune.oplabspbc.dataset_growthepie_l2_metadata
Response status code: 200
Response content: b'{"success":true,"table_name":"growthepie_l2_metadata"}'
table at: dune.oplabspbc.dataset_l2beat_l2_activity
Response status code: 200
Response content: b'{"success":true,"table_name":"l2beat_l2_activity"}'
table at: dune.oplabspbc.dataset_l2beat_l2_activity_monthly
Response status code: 200
Response content: b'{"success":true,"table_name":"l2beat_l2_activity_monthly"}'
table at: dune.oplabspbc.dataset_l2beat_l2_metadata
Response status code: 200
Response content: b'{"success":true,"table_name":"l2beat_l2_metadata"}'
